<a href="https://colab.research.google.com/github/Hing-9/popup_store/blob/main/KcElectra_Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/Graduation-Thesis/NER

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Graduation-Thesis/NER


In [ ]:
import json
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import random

In [ ]:
def file_to_data(path) :
    with open(path, 'r', encoding='UTF-8') as f :
        return json.load(f)['data']

def data_only_org(data) :
    ner_sentences = []
    for sentence in data :
        if sentence['ner'] : # NER 있는 데이터 발견
            tags = sentence['ner']['tags']
            org_flag = False
            for tag in tags :
                if tag['tag'] == 'ORG' :
                    org_flag = True
                    break
            if org_flag :
                ner_sentences.append(sentence)
    return ner_sentences

In [ ]:
data_영한 = file_to_data('./일상생활및구어체_영한_train_set.json')
data_한영 = file_to_data('./일상생활및구어체_한영_train_set.json')

In [ ]:
# 데이터 결합
data = data_영한.copy()
data.extend(data_한영.copy())

print(f'{len(data)} = {len(data_영한)} + {len(data_한영)}')

assert len(data) == len(data_영한)+len(data_한영)

2400307 = 1200307 + 1200000


In [ ]:
print(f'전체 데이터 개수 : '+str(len(data)))
data = data_only_org(data)
print(f'NER에 ORG가 포함된 데이터 개수 : '+str(len(data)))

전체 데이터 개수 : 2400307
NER에 ORG가 포함된 데이터 개수 : 30244


In [ ]:
data[0]

{'sn': 'ECOAR2T00367',
 'data_set': '일상생활및구어체',
 'domain': '해외고객과의채팅',
 'subdomain': '숙박,음식점',
 'en_original': 'I am a professor at BBB1.',
 'en': 'I am a professor at BBB1.',
 'mt': '저는 BBB1의 교수입니다.',
 'ko': '저는 BBB1의 교수입니다.',
 'source_language': 'en',
 'target_language': 'ko',
 'word_count_ko': 3.0,
 'word_count_en': 6.0,
 'word_ratio': 0.5,
 'file_name': '해외고객과의채팅_숙박,음식점.xlsx',
 'source': '크라우드 소싱',
 'license': 'open',
 'style': '구어체',
 'included_unknown_words': False,
 'ner': {'text': '저는 <ORG>BBB1의</ORG> 교수입니다.',
  'tags': [{'tag': 'ORG', 'value': 'BBB1의', 'position': '[3, 8]'}]}}

In [ ]:
# # 이거는 실행해볼 필요는 없음!!
# def kind_of_ner_tags(data) :
#     result = []
#     for sentence in data :
#         tags = sentence['ner']['tags']
#         for tag in tags :
#             result.append(tag['tag'])
    
#     result = list(set(result))
#     return result

# def value_for_tag(find_tag, data) :
#     result = []
#     MAX_CNT = 10
#     cnt = 0

#     for sentence in data :
#         tags = sentence['ner']['tags']
#         for tag in tags :
#             if tag['tag'] == find_tag :
#                 result.append(tag['value'])
#                 cnt += 1
        
#         if cnt >= MAX_CNT :
#                 break;
    
#     return result


# tags = kind_of_ner_tags(data)
# print(f'모든 데이터에서 검출된 NER TAGS : {tags}')
# print()

# for tag in tags :
#     print(f'==== TAG : {tag}====')
#     print(value_for_tag(tag, data))

In [ ]:
# # 혹시나.. 팝업이 들어간 데이터를 추출해보자
# def find_popup(data) : 
#     result = []
#     for sentence in data :
#         if '팝업' in sentence['ko'] :
#             result.append(sentence['ko'])
#         # elif '스토어' in sentence['ko'] :
#         #     result.append(sentence['ko'])
#     return result

# find_popup(data)

In [ ]:
# ORG 태그에 브랜드명을 넣어 학습시키는게 정당해보임.
# 브랜드명은 소상공인 데이터
brand_data = pd.read_csv('./소상공인시장진흥공단_상가(상권)정보_서울_202212.csv', encoding='UTF-8')
brand_names = brand_data['상호명'].values

In [ ]:
# 랜덤하게 브랜드명을 입력
rnd_idx = random.randrange(0, len(brand_names))
brand_names[rnd_idx]

'아모레카운셀러'

In [ ]:
def random_brand_transfer(raw_data) :
    data = raw_data.copy()
    tokens_arr = []
    tags_arr  = []

    for idx, sentence in enumerate(data) :

        # NER 없는 데이터면 Pass
        if not sentence['ner'] :
            continue
        
        tokens_sentence = []
        tags_sentence = []

        old_org_list = []
        chg_org_list = []

        tags = sentence['ner']['tags']

        # print(f'{idx} : {sentence["ko"]}')

        for tag in tags :
            if tag['tag'] == 'ORG' :
                
                # 브랜드 이름 랜덤 추출
                rnd_idx = random.randrange(0, len(brand_names))
                chg_org_list.append(brand_names[rnd_idx])

                # 전체 문장 치환
                # start_index = sentence['ko'].index(tag['value'])
                # end_index = start_index+len(tag['value'])
                # tag['value'] = chg_org
                positions = tag['position'][1:-1].split(',')
                start_index = int(positions[0])
                end_index = int(positions[1])
                old_org_list.append(sentence['ko'][start_index:end_index])

        if len(chg_org_list) < 1 :
            continue

        assert len(old_org_list) == len(chg_org_list)

        for i in range(len(old_org_list)) :
            old_org_nm = old_org_list[i]
            chg_org_nm = chg_org_list[i]

            target = sentence['ko']
            target = target.replace(old_org_nm, chg_org_nm)
            sentence['ko'] = target
        
        # print(f'{idx} : {sentence["ko"]}')

        # token과 tag를 Setting
        tokens_sentence = list(sentence['ko'])

        for i in range(len(tokens_sentence)) :
            tags_sentence.append('O')

        assert len(tokens_sentence) == len(tags_sentence)

        for org_name in chg_org_list :

            start_idx = -1
            while True :
                start_idx = sentence['ko'].find(org_name, start_idx+1)

                if start_idx == -1 :
                    break;
                
                end_idx = start_idx + len(org_name)

                for i in range(start_idx, end_idx) :
                    if i > start_idx :
                        tag_nm = 'I-ORG'
                    else :
                        tag_nm = 'B-ORG'
                    
                    tags_sentence[i] = tag_nm

        result_tokens_sentence = []
        for token in tokens_sentence :
            tmp = token.replace(' ','_')
            result_tokens_sentence.append(tmp)
            
        # print(tokens_sentence)
        # print(tags_sentence)
        # print(f'tokens : {len(tokens_sentence)}, tags : {len(tags_sentence)}')

        tokens_arr.append(result_tokens_sentence)
        tags_arr.append(tags_sentence)

    return tokens_arr, tags_arr

In [ ]:
tokens, tags = random_brand_transfer(data)

In [ ]:
data[2140]

{'sn': 'EISWR2F10901',
 'data_set': '일상생활및구어체',
 'domain': '해외영업',
 'subdomain': '도소매유통',
 'en_original': 'However, users singled out FFF and we decide to launched it after revamping it.',
 'en': 'However, users singled out FFF and we decide to launched it after revamping it.',
 'mt': '하지만 유저들이 FFF를 꼽았고, 저희는 FFF를 개편하여 출시하기로 결정했습니다.',
 'ko': '하지만, 사용자들이 윤기나는청소를 골랐고 저희는 윤기나는청소를 개편하여 출시하기로 했습니다.',
 'source_language': 'en',
 'target_language': 'ko',
 'word_count_ko': 9.0,
 'word_count_en': 14.0,
 'word_ratio': 0.643,
 'file_name': '해외영업_도소매유통.xlsx',
 'source': '크라우드 소싱',
 'license': 'open',
 'style': '구어체',
 'included_unknown_words': False,
 'ner': {'text': '하지만, 사용자들이 <ORG><ORG>FFF</ORG></ORG>를 골랐고 저희는 <ORG><ORG>FFF</ORG></ORG>를 개편하여 출시하기로 했습니다.',
  'tags': [{'tag': 'ORG', 'value': 'FFF', 'position': '[11, 14]'},
   {'tag': 'ORG', 'value': 'FFF', 'position': '[24, 27]'}]}}